In [16]:
import os
from google import genai
from google.genai import types

#gemma-3-27b-it
#Gemini 2.5 Flash‑Lite
class GeminiWrapper:
    def __init__(self, model_name: str = "gemini-2.5-flash-lite", api_key: str | None = None):
        self.model_name = model_name
        self.client = genai.Client(api_key=api_key or os.getenv("GEMINI_API_KEY"))

    def ask(self, question: str, system_prompt: str | None = None) -> str:
        config = None
        if system_prompt:
            config = types.GenerateContentConfig(system_instruction=system_prompt)

        response = self.client.models.generate_content(
            model=self.model_name,
            contents=question,
            config=config,
        )
        return response.text


In [ ]:
client = GeminiWrapper()  

system_prompt = "You are a concise assistant. Answer in 2-3 sentences."
question = "Why do cats purr?"

answer = client.ask(question=question, system_prompt=system_prompt)
print(answer)


Cats purr for various reasons, most commonly when they feel content and relaxed. However, they can also purr when they are injured, stressed, or in pain, as the vibrations may help them to self-soothe and promote healing.


In [2]:
import json
from pathlib import Path

cmp_path = Path('dist_check_comparison.json')
if not cmp_path.exists():
    raise FileNotFoundError('Run distribution_check.py first to create dist_check_comparison.json')

data = json.loads(cmp_path.read_text(encoding='utf-8'))

def print_table(title, rows_dict):
    print(f'\n## {title}')
    item_w = max(24, min(50, max(len(str(k)) for k in rows_dict))) if rows_dict else 24
    print(f"{'item':<{item_w}} | {'desired':>10} | {'achieved':>10} | {'status':^6}")
    print(f"{'-' * item_w}-+-{'-' * 10}-+-{'-' * 10}-+-{'-' * 6}")
    for item, row in rows_dict.items():
        label = str(item)
        if len(label) > item_w:
            label = label[: item_w - 1] + '…'
        print(f"{label:<{item_w}} | {row['desired']:>10.4f} | {row['achieved']:>10.4f} | {row['status']:^6}")

flat_sections = [
    'scenario',
    'sub_scenario_global',
    'complexity',
    'outcome',
    'conflict_level',
    'mistakes_present',
    'num_mistakes_if_present',
    'hidden_dissatisfaction_given_resolved',
]

for section in flat_sections:
    print_table(section, data['comparison'][section])

print('\n## sub_scenario_within_scenario')
for scenario, rows in data['comparison']['sub_scenario_within_scenario'].items():
    print_table(f'sub_scenario_within_scenario::{scenario}', rows)



## scenario
item                     |    desired |   achieved | status
-------------------------+------------+------------+-------
account_access           |     0.1500 |     0.1480 |   🟩   
payment_issue            |     0.2500 |     0.2320 |   🟩   
refund_request           |     0.1000 |     0.0900 |   🟩   
tariff_question          |     0.3000 |     0.3020 |   🟩   
technical_issue          |     0.2000 |     0.2280 |   🟩   

## sub_scenario_global
item                                   |    desired |   achieved | status
---------------------------------------+------------+------------+-------
2FA code not received                  |     0.0214 |     0.0200 |   🟩   
account locked                         |     0.0214 |     0.0200 |   🟩   
app does not open                      |     0.0286 |     0.0300 |   🟩   
auto-renewal                           |     0.0429 |     0.0400 |   🟩   
cannot change email                    |     0.0214 |     0.0280 |   🟩   
charge after subscription

In [1]:
import generate
from distribution_check import build_targets_manifest, compute_achieved, build_comparison

RUNS = 100
N_PER_RUN = 500
BASE_SEED = 42
MAX_RETRIES = 3

def build_dummy_rows_for_run(n, seed, max_retries=3):
    rng = generate.make_rng(seed)
    rows = []
    failed = []
    for i in range(n):
        dialogue_id = f'dlg_{i:06d}'
        spec = generate.sample_generation_spec(rng, dialogue_id)
        wrote = False
        reasons = []
        for attempt in range(max_retries + 1):
            messages = generate.generate_messages_from_spec(rng, spec, attempt=attempt, dummy_text=True)
            ok, reasons = generate.validate_messages(messages, spec['complexity'])
            if ok:
                rows.append(generate.assemble_record(spec, messages))
                wrote = True
                break
        if not wrote:
            failed.append({'dialogue_id': dialogue_id, 'reasons': reasons})
    return rows, failed

def _avg_flat_dict(dicts):
    if not dicts:
        return {}
    keys = sorted(set().union(*[d.keys() for d in dicts]))
    return {k: sum(float(d.get(k, 0.0)) for d in dicts) / len(dicts) for k in keys}

def average_achieved(achieved_runs):
    avg = {
        'n_records': sum(r['n_records'] for r in achieved_runs) / len(achieved_runs),
        'scenario': _avg_flat_dict([r['scenario'] for r in achieved_runs]),
        'sub_scenario_global': _avg_flat_dict([r['sub_scenario_global'] for r in achieved_runs]),
        'sub_scenario_within_scenario': {},
        'complexity': _avg_flat_dict([r['complexity'] for r in achieved_runs]),
        'outcome': _avg_flat_dict([r['outcome'] for r in achieved_runs]),
        'conflict_level': _avg_flat_dict([r['conflict_level'] for r in achieved_runs]),
        'mistakes_present': _avg_flat_dict([r['mistakes_present'] for r in achieved_runs]),
        'num_mistakes_if_present': _avg_flat_dict([r['num_mistakes_if_present'] for r in achieved_runs]),
        'hidden_dissatisfaction_given_resolved': _avg_flat_dict([r['hidden_dissatisfaction_given_resolved'] for r in achieved_runs]),
        'counts': {
            'resolved_total': sum(r['counts']['resolved_total'] for r in achieved_runs) / len(achieved_runs),
            'mistakes_present_total': sum(r['counts']['mistakes_present_total'] for r in achieved_runs) / len(achieved_runs),
        },
    }
    scenarios = sorted(set().union(*[r['sub_scenario_within_scenario'].keys() for r in achieved_runs]))
    for scenario in scenarios:
        avg['sub_scenario_within_scenario'][scenario] = _avg_flat_dict([
            r['sub_scenario_within_scenario'].get(scenario, {}) for r in achieved_runs
        ])
    return avg

targets = build_targets_manifest()
all_run_results = []  # best structure here: list[dict] (easy to inspect/filter in notebook)
all_achieved = []
all_failures = []

for run_idx in range(RUNS):
    seed = BASE_SEED + run_idx
    rows, failed = build_dummy_rows_for_run(N_PER_RUN, seed, MAX_RETRIES)
    achieved = compute_achieved(rows)
    comparison = build_comparison(targets, achieved)
    all_achieved.append(achieved)
    all_failures.append({'run_idx': run_idx, 'seed': seed, 'failed_count': len(failed)})
    all_run_results.append({
        'run_idx': run_idx,
        'seed': seed,
        'achieved': achieved,
        'summary': comparison['summary'],
    })

average_achieved_result = average_achieved(all_achieved)
average_comparison = build_comparison(targets, average_achieved_result)

def print_table(title, rows_dict):
    print(f'\n## {title}')
    item_w = max(24, min(50, max(len(str(k)) for k in rows_dict))) if rows_dict else 24
    print(f"{'item':<{item_w}} | {'desired':>10} | {'achieved':>10} | {'status':^6}")
    print(f"{'-' * item_w}-+-{'-' * 10}-+-{'-' * 10}-+-{'-' * 6}")
    for item, row in rows_dict.items():
        label = str(item)
        if len(label) > item_w:
            label = label[: item_w - 3] + '...'
        print(f"{label:<{item_w}} | {row['desired']:>10.4f} | {row['achieved']:>10.4f} | {row['status']:^6}")

print(f'Runs: {RUNS}, rows/run: {N_PER_RUN}, total rows simulated: {RUNS * N_PER_RUN}')
print(f"Avg failed rows per run: {sum(x['failed_count'] for x in all_failures) / len(all_failures):.2f}")

flat_sections = [
    'scenario',
    'sub_scenario_global',
    'complexity',
    'outcome',
    'conflict_level',
    'mistakes_present',
    'num_mistakes_if_present',
    'hidden_dissatisfaction_given_resolved',
]
for section in flat_sections:
    print_table(f'AVERAGE::{section}', average_comparison['comparison'][section])

print('\n## AVERAGE::sub_scenario_within_scenario')
for scenario, rows in average_comparison['comparison']['sub_scenario_within_scenario'].items():
    print_table(f'AVERAGE::sub_scenario_within_scenario::{scenario}', rows)


Runs: 100, rows/run: 500, total rows simulated: 50000
Avg failed rows per run: 0.00

## AVERAGE::scenario
item                     |    desired |   achieved | status
-------------------------+------------+------------+-------
account_access           |     0.1500 |     0.1499 |   🟩   
payment_issue            |     0.2500 |     0.2520 |   🟩   
refund_request           |     0.1000 |     0.0996 |   🟩   
tariff_question          |     0.3000 |     0.2983 |   🟩   
technical_issue          |     0.2000 |     0.2002 |   🟩   

## AVERAGE::sub_scenario_global
item                                   |    desired |   achieved | status
---------------------------------------+------------+------------+-------
2FA code not received                  |     0.0214 |     0.0225 |   🟩   
account locked                         |     0.0214 |     0.0227 |   🟩   
app does not open                      |     0.0286 |     0.0284 |   🟩   
auto-renewal                           |     0.0429 |     0.0423 |   🟩 